In [1]:
%load_ext autoreload
%autoreload 2

In [38]:
from simple_radio_script_encapsulated import _get_teaser_for_story, _get_statements_for_story_id
from simple_radio_script_encapsulated import teaser_matches_crit, aggregate_decisions

In [39]:
story_no = 23188

In [40]:
t1 = _get_teaser_for_story(story_no)
t1

'Die Computertomographie (CT) ist eines der wichtigsten Diagnoseverfahren in der Medizin. Der Nutzen ist unumstritten. Gleichwohl ist auch bekannt, dass zu viel Röntgenstrahlung das Krebsrisiko erhöhen kann. Je jünger die Patienten, desto wichtiger ist ein Blick auf diese Dosis-Wirkungs-Beziehung. Jedes Jahr unterziehen sich mehr als eine Million Kinder in Europa einem CT-Scan. Wie groß das Strahlenrisiko bei ihnen genau ist, ist nicht bekannt.\n\nIn einer großen Studie im Fachblatt „Nature Medicine“ hat nun ein internationales Forschungsteam fast eine Million Menschen in neun europäischen Ländern untersucht, die vor ihrem 22. Lebensjahr CT-Scans hatten (siehe Primärquelle). Die Autorinnen und Autoren berechneten zunächst die Strahlenbelastung der Patienten. Dabei zeigte sich ein Zusammenhang zwischen der Menge an Strahlung und dem Risiko für Blutkrebs. Für jede Person wurde die Strahlendosis geschätzt, die auf das Knochenmark einwirkt, wo Blutzellen produziert werden. Bei Leukämie kom

In [41]:
statements = _get_statements_for_story_id(story_no)
print(statements)

[{'expert_name': 'Prof. Dr. Hajo Zeeb', 'statement': '„Das ist die lang erwartete Epi-CT-Studie, in die auch unsere (wenigen) deutschen Daten eingegangen sind. Hauptsächlich wird die Datenmenge aber von der großen Studie aus Großbritannien geprägt (rund 50 Prozent), die Daten der ersten Veröffentlichungen dieser Studie wurden allerdings für die vorliegende Analyse nochmal aktualisiert und erweitert.“\n\n„Da viele Einzelkohorten schon vorher publiziert wurden, überrascht das Gesamtergebnis erhöhter Risiken pro Dosis für hämatologische Krebserkrankungen bei jungen Menschen nicht. Einige Auffälligkeiten gibt es aber schon. Zum Beispiel die erhöhten Risiken für das Hodgkin-Lymphom, was in der großen Referenzstudie in der Strahlenepidemiologie, der Life-Span-Study bei Überlebenden der Atombombenabwürfe in Japan, so nicht gefunden wird, und auch nicht in den bisherigen Einzelveröffentlichungen zu CT-Expositionen. Dies ist aber nur ein kleiner Anteil aller Fälle, und muss in Zukunft weiter un

In [46]:
crit_match_prompt = """
    You are a professional journalist in the medicine domain. You will receive a text and i want you to decide whether the following criteria matches the text.
    To help you guide your decision, you will also get a few expert statements in JSON Form.
    Please answer in the following format.
    The Criteria is: {CRITERIA}
    Decision: yes or no. Reference: Paragraph in the text which matches the criteria.

    ```
    TEXT: {TEASER}
    ```
    ```
    EXPERT STATEMENTS: 
    {STATEMENTS}
    ```
    Please answer in German.
    """ 

In [62]:
agg_decisions_prompt = """
    I will provide a list of Texts to you. These Texts are decisions for specified criteria on a news article. 
    I want you to build a table with the columns criteria, match and reference.
    The column critera should name the specified criteria, match indicates if the criteria matches or not and the column reference shows the part of the text which corresponds to the criteria

    The different decisions are separated by the | symbol

    Here is the list of the decisions:
    ```
    TEXT: {TEXT}
    ```
    Please answer in german.

"""

In [58]:
crit1 = "The text mentions one or more persons explicitly"
crit2 = "The text contains information about cancer"
crit3 = "The text contains mentions a specifoc journal name of a study"
crit4 = "The text describes a mathematical method for optimizing a loss function with four objectives"

crits = [crit1, crit2, crit3, crit4]

In [49]:
outputs = [teaser_matches_crit(t1, str(statements), crit_match_prompt, c) for c in crits]

In [50]:
outputs

['Die Kriterien sind: Der Text erwähnt eine oder mehrere Personen explizit.\nEntscheidung: Nein. Referenz: Im Text werden keine spezifischen Personen namentlich erwähnt.',
 'Die Kriterien sind: Der Text enthält Informationen über Krebs.\nEntscheidung: Ja. Referenz: Im Text wird ein Zusammenhang zwischen der Menge an Strahlung durch CT-Scans und dem Risiko für Blutkrebs, insbesondere Leukämie, beschrieben. Dies wird im zweiten Absatz deutlich: "Dabei zeigte sich ein Zusammenhang zwischen der Menge an Strahlung und dem Risiko für Blutkrebs."',
 'Die Kriterien sind: Der Text enthält die Erwähnung eines spezifischen Zeitschriftennamens einer Studie.\nEntscheidung: Ja. Referenz: Im Text wird im zweiten Absatz das Fachblatt „Nature Medicine“ erwähnt, in dem die Studie veröffentlicht wurde: "In einer großen Studie im Fachblatt „Nature Medicine“ hat nun ein internationales Forschungsteam fast eine Million Menschen in neun europäischen Ländern untersucht, die vor ihrem 22. Lebensjahr CT-Scans h

In [63]:
table_out = aggregate_decisions(outputs, agg_decisions_prompt)

In [ ]:
file_path='output.md'
with open(file_path, 'w') as file:  
    print(table_out)  
    print(table_out, file = file)

{{ table_out }}